In [20]:
import numpy as np
import pandas as pd
import math
import csv
import openpyxl
'''train = np.array([
    [0,0,1,2,3],
    [1,1,2,3,4],
    [2,2,3,4,5],
    [3,3,4,5,6]
]
)'''
'''
train = np.array([
    [0,1,1,1,3],
    [1,1,1,1,4],
    [2,1,1,1,5],
    [3,1,1,1,6]
]
)'''
train = pd.read_csv('train.csv').as_matrix()#read train.csv file
#print(type(train))
#xtrain=train[:,1:4]
#xtrain = xtrain/5
#true_label=train[:,4:5]
#print((xtrain),(true_label))


C:\Users\LN Pandey\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [21]:
def encode_into_prob(x,max_val):
    len=x.size
    y_list=[]
    for i in range(len):
        temp=[]
        for j in range(max_val):
            if(x[i]==j):
                temp.append(1)
            else:
                temp.append(0)
            #print(temp)
        y_list.append(temp)
    return y_list
        

In [22]:
xtrain=train[:,1:785]

xmean = np.mean(xtrain , axis=1)
means_expanded = np.outer(xmean, np.ones(784))
#print(means_expanded)
xtrain = xtrain - means_expanded

xstd = np.std(xtrain , axis=1)
std_expanded = np.outer(xstd, np.ones(784))
#print(std_expanded)
xtrain = xtrain*1.0/std_expanded 

true_label = train[:,785:]
y_true=np.array(encode_into_prob(true_label,10))
y_true=y_true.transpose()

#print(type(y_true))
#print(y_true.transpose())
print(xtrain)
print((y_true))

[[-0.55945675 -0.55945675  2.06254185 ...  2.0522595  -0.55945675
  -0.55945675]
 [-0.5380259  -0.5380259  -0.5380259  ... -0.5380259  -0.5380259
  -0.5380259 ]
 [-0.6486501  -0.6486501  -0.6486501  ... -0.6486501  -0.6486501
  -0.6486501 ]
 ...
 [-0.64973918 -0.64973918 -0.64973918 ... -0.64973918 -0.64973918
  -0.64973918]
 [-0.59745866 -0.59745866 -0.59745866 ... -0.59745866 -0.59745866
  -0.59745866]
 [-0.62629522 -0.62629522 -0.62629522 ... -0.62629522 -0.62629522
  -0.62629522]]
[[0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]]


In [23]:
def feedforward(batch_train,hidden_layers):
     
    global weight_matrix_list
    global bias_list
    global preactivation_list 
    preactivation_list=[]
    global activation_list
    activation_list=[]
    global y_hat
    y_hat=[]
    a_new=[]
    a_last_layer=[]
    nos_data = len(batch_train)
    #print(nos_data)
    h=batch_train.transpose()
    preactivation_list.append(h)
    activation_list.append(h)
    #print(h)
    
#     print("y_hat in feed\n",y_hat )
    
    for j in range(hidden_layers):
        
#         print(".............................")
#         print("activation for layer ", j)
#         print(activation_list[j] , activation_list[j].shape)
#         print("weight",j+1)
#         print(weight_matrix_list[j], weight_matrix_list[j].shape)
#         print("bias ",j+1)
#         print(bias_list[j],bias_list[j].shape)
        
        a_new = bias_list[j]+np.matmul(weight_matrix_list[j].transpose(),activation_list[j])
        preactivation_list.append(a_new)
        
#         print("preactivation for layer",j+1)
#         print(preactivation_list[j+1], preactivation_list[j+1].shape)
        
        h = 1.0/(1.0+np.exp(-1*preactivation_list[j+1]))
        activation_list.append(h)

    
    j=hidden_layers
#     print(".............................")
#     print("activation for layer ", j)
#     print(activation_list[j] , activation_list[j].shape)
#     print("weight",j+1)
#     print(weight_matrix_list[j], weight_matrix_list[j].shape)
#     print("bias ",j+1)
#     print(bias_list[j],bias_list[j].shape)
    
    a_last_layer = bias_list[j]+np.matmul(weight_matrix_list[j].transpose(),activation_list[j])
    preactivation_list.append(a_last_layer)
    
#     print("last activation")
#     print(a_last_layer,a_last_layer.shape)

    y_hat = np.array(output(a_last_layer,batch_train))
    y_hat = y_hat.transpose()
    
#     print("y_hat in feedforward")
#     print(y_hat, y_hat.shape)
    
    
def output(a_last_layer,batch_train):
    y_hat_initial=[]
    y_hat_local=[]
    nos_data=len(batch_train)
    for i in range(nos_data):
        y_hat_initial.append(softmax(a_last_layer[:,i:i+1].transpose()))

    length = len(y_hat_initial)
    for i in range(length):
        y_hat_local.append(y_hat_initial[i][0])
    
#     print("y_hat_local", y_hat_local)
#     print("y_hat_initial",y_hat_initial)
    return y_hat_local

def softmax(row_vector):
    sum_all=0
    row_vector = np.exp(row_vector)
    sum_all = np.sum(row_vector , axis=1)         #print("row_vector", row_vector)
    row_vector = (row_vector*1.0/sum_all)         #print("sum_all",sum_all)
    return row_vector

In [24]:
def backpropagation(y_true_batch):
    
#     print("weigt in backpropogation", weight_matrix_list)
#     print("...........y_true_batch..........\n",y_true_batch)
    
    global grad_a_k_list
    grad_a_k_list=[]
    
    global grad_w_k_list
    grad_w_k_list=[]
    
    global grad_h_k_list
    grad_h_k_list=[]
    
    global grad_b_k_list
    grad_b_k_list=[]
    
    global grad_b
    grad_b=[]
    
    global grad_g_dash_a_k_list
    grad_g_dash_a_k_list = []
    
    global grad_a_last
    grad_a_last=[]
    
    k = num_hidden+1
    
#     print("--------------------------in Backpropagation---------------------")
    
    grad_g_dash_a_k_list = g_dash(preactivation_list)
    
#     print("preactivation_list....................")
#     print("Nos. of matrices in preactivation_list is", len(preactivation_list))
#     print(preactivation_list)
#     print("g_dash..................")
#     print("Nos. of matrices in g_dash is ", len(grad_g_dash_a_k_list))
#     print(grad_g_dash_a_k_list)
    
    grad_a_last = -np.subtract(y_true_batch,y_hat)
    
#     print("y_true_batch \n", y_true_batch)
#     print("y_hat \n",y_hat)
    
    grad_a_k_list.insert(0,grad_a_last)
    
#     print("grad_a_last_layer", k)
#     print(grad_a_k_list[0],grad_a_k_list[0].shape)
        
    while(k>0):
        
#         print(".................grad_compute at layer",k)
        
        grad_w_k_list.insert(0,np.matmul(grad_a_k_list[0],activation_list[k-1].transpose()))
        
#         print("grad_a", k)
#         print(grad_a_k_list[0],grad_a_k_list[0].shape)
#         print('preactivation  a_',k-1)
#         print(preactivation_list[k-1],preactivation_list[k-1].shape)
#         print("grad_w_",k)
#         print(grad_w_k_list[0],grad_w_k_list[0].shape)
        
        grad_b_k_list.insert(0,grad_a_k_list[0])
        
#         print("grad_b",k)
#         print(grad_b_k_list[0],grad_b_k_list[0].shape)
        
        grad_h_k_list.insert(0,np.matmul(weight_matrix_list[k-1],grad_a_k_list[0]))
        
#         print("grad_w",k)
#         print(grad_w_k_list[0],grad_w_k_list[0].shape)
#         print("grad_a",k)
#         print(grad_a_k_list[0],grad_a_k_list[0].shape)
#         print("grad_h",k-1)
#         print(grad_h_k_list[0],grad_h_k_list[0].shape)
        
        grad_a_k_list.insert(0,np.multiply(grad_h_k_list[0],grad_g_dash_a_k_list[k-1]))
        
#         print("grad_h_k_list[0]",k-1)
#         print(grad_h_k_list[0],grad_h_k_list[0].shape)
#         print("grad_g_dash_a_k_list[k-1]",k-1)
#         print(grad_g_dash_a_k_list[k-1],grad_g_dash_a_k_list[k-1].shape)
#         print("grad_a",k-1)
#         print(grad_a_k_list[0],grad_a_k_list[0].shape)
        k=k-1
#         print("grad_a_ik_list",grad_a_k_list[0])
    
    sum_all_grad_b_of_data(grad_b_k_list)
    

def sum_all_grad_b_of_data(grad_b_k_list):
    length=len(grad_b_k_list)
    i=0
    while(i<length):
        row=[]
        row = np.sum(grad_b_k_list[i],1)
        j = len(row)
        #print(row.shape)
        row = row[:, np.newaxis]
        #print(row.shape , row)
        grad_b.append(row)
        i+=1
    #print("grad_b",grad_b)

def g_dash(preactivation_list):
    length=len(preactivation_list)
    a_modified = preactivation_list[0:length]
    g_dash_a_k_list=[]
    for i in range(length):
        a=(1.0/(1+np.exp(-a_modified[i])))*(1-(1.0/(1+np.exp(-a_modified[i]))))
        g_dash_a_k_list.append(a)
    return g_dash_a_k_list


In [25]:
def decode_yhat_to_classes(y_hat_local):
    col = len(y_hat_local[0])
    max_val=0
    for j in range(col):
        max_val = y_hat_local[0][j]
        index=0
        for i in range(9):
            if(y_hat_local[i+1][j] > max_val):
                y_hat_local[index][j]=0
                index = i+1
                max_val=y_hat[i+1][j]
            else :
                y_hat_local[i+1][j]=0
        y_hat_local[index][j]=1
    return y_hat_local

In [54]:
#ip_neurons = 3 #input layer
ip_neurons = 784
num_hidden = 0
encoding_bits=10
sizes = []
sizes.insert(0,ip_neurons)
sizes.append(encoding_bits)
print(sizes)
print(len(sizes))

weight_matrix_list = []

bias_list =[]
y_hat= []

for i in range(int(num_hidden)+1):
    weight_matrix_list.append(np.random.rand(sizes[i],sizes[i+1])/np.sqrt(sizes[i]))
    bias_list.append(0*np.random.rand(sizes[i+1],1))
print(weight_matrix_list)
#print(len(weight_matrix_list))

print(bias_list)
#print(hlist)



[784, 10]
2
[array([[0.00140621, 0.00970851, 0.00100002, ..., 0.00474145, 0.00426977,
        0.0046401 ],
       [0.00824713, 0.00351868, 0.00859146, ..., 0.00435939, 0.00077917,
        0.00614494],
       [0.00994193, 0.0077316 , 0.00409722, ..., 0.00948514, 0.00735271,
        0.00888722],
       ...,
       [0.00531261, 0.00191074, 0.00964108, ..., 0.00280564, 0.0053278 ,
        0.00022512],
       [0.00295578, 0.00769845, 0.00605919, ..., 0.00982787, 0.00670437,
        0.00929958],
       [0.00471208, 0.00848609, 0.00222295, ..., 0.00265812, 0.00867386,
        0.00645128]])]
[array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])]


In [55]:
def adam(weight_matrix_list,bias_list,xtrain,num_hidden,sizes):
    
    m_w = [] 
    m_b = []
    for i in range(int(num_hidden)+1):
        m_w.append(0*np.random.rand(sizes[i],sizes[i+1]))
        m_b.append(0*np.random.rand(sizes[i+1],1))
    
    v_w = [] 
    v_b =[]
    for i in range(int(num_hidden)+1):
        v_w.append(0*np.random.rand(sizes[i],sizes[i+1]))
        v_b.append(0*np.random.rand(sizes[i+1],1))
        
    len_w = len(weight_matrix_list)  
    len_b = len(bias_list)
    
    eta = 0.01
    batch_size = 10
    num_points_seen = 0
    beta1 , beta2 , eps = 0.9 , 0.999 , 1e-8
    epochs = 20
    t=0
    data_length = len(xtrain)
    no_of_batch = int(data_length/batch_size)
    
    for t in range(epochs):
        for batch in range(no_of_batch):
            dw=[]
            db=[]
            for i in range(int(num_hidden)+1):
                dw.append(0*np.random.rand(sizes[i],sizes[i+1]))
                db.append(0*np.random.rand(sizes[i+1],1))
            
            for i in range(batch_size):
                l = batch_size*batch + i
                r = batch_size*batch + i + 1
                feedforward(xtrain[l:r,:],num_hidden)
                #print(l , r)
                backpropagation(y_true[:,l:r])

                for j in range(len_w): 
                    dw[j] += grad_w_k_list[j].transpose()
                                    
                for j in range(len_b):
                    db[j] += grad_b[j]
                                
            for j in range(len_w): 
                m_w[j] = beta1*m_w[j] + (1-beta1)*dw[j]
                                    
            for j in range(len_b):
                m_b[j] = beta1*m_b[j] + (1-beta1)*db[j]    
            
            for j in range(len_w): 
                v_w[j] = beta2*v_w[j] + (1-beta2)*(dw[j]**2)
                                    
            for j in range(len_b):
                v_b[j] = beta2*v_b[j] + (1-beta2)*(db[j]**2)
                
            for j in range(len_w): 
                m_w[j] = m_w[j]/(1-math.pow(beta1 , batch + 1))
                                    
            for j in range(len_b):
                m_b[j] = m_b[j]/(1-math.pow(beta1 , batch + 1))
             
            for j in range(len_w): 
                v_w[j] = v_w[j]/(1-math.pow(beta2 , batch + 1))
                                    
            for j in range(len_b):
                v_b[j] = v_b[j]/(1-math.pow(beta2 , batch + 1))
                
            for j in range(len_w):
                weight_matrix_list[j] -= (eta/np.sqrt(v_w[j] + eps))*m_w[j]
            
            for j in range(len_b):
                bias_list[j] -= (eta/np.sqrt(v_b[j] + eps))*m_b[j]
        
        print("end of epoch" , t)
            
#             print("batch", batch)
#             y_hat_local = decode_yhat_to_classes(y_hat)
#             print(y_hat)
                
#         print("end  epochs :  ", t)
#         y_hat_local = decode_yhat_to_classes(y_hat)
#         print(y_hat)
#     print(bias_list)
#     print(preactivation_list)

In [56]:
adam(weight_matrix_list,bias_list,xtrain,num_hidden,sizes)
#print(y_hat)

C:\Users\LN Pandey\Anaconda3\lib\site-packages\ipykernel_launcher.py:67: RuntimeWarning: overflow encountered in true_divide
C:\Users\LN Pandey\Anaconda3\lib\site-packages\ipykernel_launcher.py:70: RuntimeWarning: overflow encountered in true_divide


end of epoch 0
end of epoch 1
end of epoch 2
end of epoch 3
end of epoch 4
end of epoch 5
end of epoch 6
end of epoch 7
end of epoch 8
end of epoch 9
end of epoch 10
end of epoch 11
end of epoch 12
end of epoch 13
end of epoch 14
end of epoch 15
end of epoch 16
end of epoch 17
end of epoch 18
end of epoch 19


In [44]:
print(y_hat)

[[0.1048692 ]
 [0.09999369]
 [0.10451354]
 [0.09872875]
 [0.09517743]
 [0.10335944]
 [0.09530957]
 [0.09939531]
 [0.10105902]
 [0.09759406]]


In [50]:
def decode_yhat_to_classes(y_hat):
    col = len(y_hat[0])
    max_val=0
    for j in range(col):
        max_val = y_hat[0][j]
        index=0
        for i in range(9):
            if(y_hat[i+1][j] > max_val):
                y_hat[index][j]=0
                index = i+1
                max_val=y_hat[i+1][j]
            else :
                y_hat[i+1][j]=0
        y_hat[index][j]=1
    return y_hat

xtest = pd.read_csv('test.csv').as_matrix()
test=xtest[:,1:]

xmean = np.mean(test , axis=1)
means_expanded = np.outer(xmean, np.ones(784))
#print(means_expanded)
test = test - means_expanded

xstd = np.std(test , axis=1)
std_expanded = np.outer(xstd, np.ones(784))
#print(std_expanded)
test = test*1.0/std_expanded 


feedforward(test,num_hidden)


C:\Users\LN Pandey\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [51]:
print(y_hat)
y_hat = decode_yhat_to_classes(y_hat)
print(y_hat)

[[0.06807082 0.06825942 0.06780937 ... 0.06785387 0.06789097 0.06811857]
 [0.13407689 0.13386117 0.13437845 ... 0.13432678 0.13428027 0.13402486]
 [0.06923263 0.06941714 0.06897735 ... 0.06902276 0.06905957 0.0692788 ]
 ...
 [0.06836437 0.06855471 0.0681053  ... 0.06815119 0.06818811 0.06841243]
 [0.06756892 0.06776089 0.06730267 ... 0.06734868 0.06738946 0.0676159 ]
 [0.09623877 0.09628915 0.09616436 ... 0.09617998 0.09619066 0.09625033]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [52]:
def classifcation(y_hat):
    y_predict=[]
    col = len(y_hat[0])
    for j in range(col):
        for i in range(10):
            if(y_hat[i][j]==1):
                y_predict.append(i)
                break
    return np.array(y_predict)

In [53]:
y_predict = classifcation(y_hat)
print((y_predict))
y_predict = y_predict[:, np.newaxis]
print((y_predict))

[3 3 3 ... 3 3 3]
[[3]
 [3]
 [3]
 ...
 [3]
 [3]
 [3]]


In [38]:
def add_id_to_y_predict(y_predict):
    ide=[]
    row=len(y_predict)
    for i in range(row):
        ide.append(i)
    ide=np.array(ide)
    ide=ide[:,np.newaxis]
    y= np.hstack((ide,y_predict))
    #print(y_predict, ide)
    return y           

In [39]:
print(y_predict)
#y_predict= np.random.randint(10, size=(10000, 1))
y_predict= add_id_to_y_predict(y_predict)
print(y_predict)
x,y = y_predict.shape
#print(x,y)

[[6]
 [6]
 [1]
 ...
 [1]
 [6]
 [1]]
[[   0    6]
 [   1    6]
 [   2    1]
 ...
 [9997    1]
 [9998    6]
 [9999    1]]


In [40]:
with open('predict_from_adam.csv', 'w', newline='') as f:
    thewriter = csv.writer(f)
    row,col=y_predict.shape 
    thewriter.writerow(['id','label'])
    for i in range(row):
        thewriter.writerow(y_predict[i])
#out.close()
#np.savetxt("foo.csv", y_predict, delimiter=",")

In [ ]:
print(len(y_hat[0]))
max(3,5)

In [ ]:
print(grad_b)

In [ ]:
print(preactivation_list[3])
print(activation_list)

In [ ]:
def test_func(weight_matrix_list,bias_list,test,hidden_layers):
    nos_data = len(test)
    #print(nos_data)
    #preactivation_list_new=[]
    #activation_list_new=[]
    #for i in range(nos_data):
    h = test.transpose()
    #preactivation_list_new.append(h)
    #activation_list_new.append(h)
    #print(h)
    for j in range(hidden_layers):
        a_new = bias_list[j]+np.matmul(weight_matrix_list[j].transpose(),h)
        #preactivation_list_new.append(a_new)
        h = 1.0/(1.0+np.exp(-a_new))
        #activation_list_new.append(h)
        #print("next activation", h)

    j=hidden_layers
    #hlist.append(h)
    a_last = bias_list[j]+np.matmul(weight_matrix_list[j].transpose(),h)
    #preactivation_list.append(a_last)
    print("last activation", a_last)
    #y_hat_new = np.array(output(a_last))
    #y_hat_new = y_hat_new.transpose()
    #print("y_hat", y_hat)
    #print("y_hat_global", y_hat_global)
    #print(y_hat.transpose())
    return y_hat

In [ ]:
test = np.array([
    [5,6,7],
    [6,7,8]
]
)
y=test_func(weight_matrix_list,bias_list,test,num_hidden)
print(y)

In [ ]:
x= np.array([
    [0,0,1,2,3],
    [1,1,2,3,4],
    [2,2,3,4,5],
    [3,3,4,5,6]
]
)
y = np.array([
    [0,0,1,2,3],
    [1,1,2,3,4],
    [2,2,3,4,5],
    [3,3,4,5,6]
]
)
print(np.subtract(x,y))
print(type(x))

In [ ]:
print(type(y_true), type(y_hat))

In [ ]:
a=[1,2,3,4]
print((a[0:3]))

In [ ]:
sample = np.array([
    [0,0,1,2,3],
    [1,1,2,3,4],
    [2,2,3,4,5],
    [3,3,4,5,6]
]
)

In [ ]:
s=np.tile(np.arange(1,10,10),(10,4))
print(s)
s[:,i:i+1]

In [ ]:
y=output(s)
print(y)

In [ ]:
s=np.tile(np.arange(1,10,10),(10,4))
print(s)
print(s[:,i:i+1])
p=softmax(s[:,i:i+1].transpose())
print("P", p)

In [ ]:
print(preactivation_list)

In [ ]:
g_dash(preactivation_list)

In [461]:
x = np.array([
    [0,0,1,2,3],
    [1,1,2,3,4],
    [2,2,3,4,5],
    [3,3,4,5,6]
])
#print(type(x))
y = np.mean(x , axis=1)
means_expanded = np.outer(y, np.ones(5))
print(means_expanded)
z=x-means_expanded
a = np.std(x,axis=1)
variance_extended = np.outer(a,np.ones(5))
print(a)
print(variance_extended)
unit_x= x/variance_extended
print(unit_x)
check = np.std(unit_x, axis=1)
print(check)

[[1.2 1.2 1.2 1.2 1.2]
 [2.2 2.2 2.2 2.2 2.2]
 [3.2 3.2 3.2 3.2 3.2]
 [4.2 4.2 4.2 4.2 4.2]]
[1.16619038 1.16619038 1.16619038 1.16619038]
[[1.16619038 1.16619038 1.16619038 1.16619038 1.16619038]
 [1.16619038 1.16619038 1.16619038 1.16619038 1.16619038]
 [1.16619038 1.16619038 1.16619038 1.16619038 1.16619038]
 [1.16619038 1.16619038 1.16619038 1.16619038 1.16619038]]
[[0.         0.         0.85749293 1.71498585 2.57247878]
 [0.85749293 0.85749293 1.71498585 2.57247878 3.4299717 ]
 [1.71498585 1.71498585 2.57247878 3.4299717  4.28746463]
 [2.57247878 2.57247878 3.4299717  4.28746463 5.14495755]]
[1. 1. 1. 1.]


In [336]:
mat = np.array([
    [0,0,1,2,7],
    [1,1,2,3,4],
    [2,2,3,4,5],
    [3,3,4,5,6]
])
sum_col = np.sum(mat[0:1,:],axis=1)
print(sum_col)

[10]


In [ ]:
def output(a_last_layer,batch_train):
    y_hat_initial=[]
    y_hat_local=[]
    nos_data=len(batch_train)
    for i in range(nos_data):
        y_hat_initial.append(softmax(a_last_layer[:,i:i+1].transpose()))

    length = len(y_hat_initial)
    for i in range(length):
        y_hat_local.append(y_hat_initial[i][0])
    
    print("y_hat_local", y_hat_local)
    print("y_hat_initial",y_hat_initial)
    return y_hat_local

In [ ]:
def softmax(row_vector):
    i = row_vector.size
    sum_all=0
    row_vector = np.exp(row_vector)
    for j in range(i):
        sum_all+= row_vector[0][j]
    row_vector = (row_vector*1.0/sum_all)
    return row_vector

In [ ]:
def grad_descent(weight_matrix_list,bias_list,xtrain,num_hidden,sizes):
    t=0
    max_iteration=10
    
    eta=0.01
    data_length = len(xtrain)
    batch_size = 10
    #batch_size = data_length
    no_of_batch = int(data_length/batch_size)
    #no_of_batch =1
    while(t < max_iteration):
        dw=[]
        db=[]
        
        for i in range(int(num_hidden)+1):
            dw.append(0*np.random.rand(sizes[i],sizes[i+1]))
            db.append(0*np.random.rand(sizes[i+1],1))
            
        
        
        for batch in range(no_of_batch):
            l=batch_size*batch
            r=batch_size*batch+batch_size
            feedforward(xtrain[l:r,:],num_hidden)
            backpropagation(y_true[:,l:r])
            
            length=len(weight_matrix_list)
            i=0
            #print("grad_w",grad_w_k_list)
            #print("a_matrix",preactivation_list)
            #print("hactivation",activation_list)
            #print("hat",y_hat)
            
            while(i<length):
                dw[i] += grad_w_k_list[i].transpose()
                i+=1
                #print("weight_matrix_list", weight_matrix_list[i])
                #print("grad_w_k_list", grad_w_k_list[i])
                #print("weight_matrix_list", weight_matrix_list[i])
                
            length=len(bias_list)
            i=0
            
            while(i<length):
                db[i] += grad_b[i]
                i+=1
                #print("y_hat", y_hat)
                #y_hat_local = decode_yhat_to_classes(y_hat)
                #if(batch == batch_size-1):
                #print("y_hat", y_hat)
                #print("y_hat for batch ", batch , y_hat_local)
        
        length=len(weight_matrix_list)
        i=0
        while(i<length):
            weight_matrix_list[i] -= eta*dw[i]
            i+=1
        
        length=len(bias_list)
        i=0        
        while(i<length):
            bias_list[i] -= eta*db[i]
            i+=1
        t+=1
        print("end  epochs :  ", t)
        #print(y_hat)
#     print(bias_list)
#     print(preactivation_list)

In [ ]:
dw=[]
db=[]
        
for i in range(int(num_hidden)+1):
    dw.append(0*np.random.rand(sizes[i],sizes[i+1]))
    db.append(0*np.random.rand(sizes[i+1],1))
print(dw)
print(db)

In [3]:
import numpy as np
mat = np.array([
    [0,0,1,2,7],
    [1,1,2,3,4],
    [2,2,3,4,5],
    [3,3,4,5,6]
])
y = mat**2
print(y)

[[ 0  0  1  4 49]
 [ 1  1  4  9 16]
 [ 4  4  9 16 25]
 [ 9  9 16 25 36]]
